In [5]:
import os
from trino.dbapi import connect
import pandas as pd
from dotenv import load_dotenv

load_dotenv(
    os.path.dirname(
        os.getcwd()
    ) + '/.env'
)


True

In [6]:
conn = connect(
    host=os.getenv('TRINO_HOST'),
    port="8180",
    user=os.getenv('TRINO_USER'),
)
cur = conn.cursor()

In [7]:
# create schema
cur.execute("""
CREATE SCHEMA iceberg.bronze
WITH (location = 's3a://lakehouse/bronze')
""")
cur.execute("""
CREATE SCHEMA iceberg.silver
WITH (location = 's3a://lakehouse/silver')
""")
cur.execute("""
CREATE SCHEMA iceberg.gold
WITH (location = 's3a://lakehouse/gold')
""")

In [11]:
# create table for silver layer
cur.execute("""
CREATE TABLE iceberg.silver.asin_info (
    asin VARCHAR,
    country VARCHAR,
    product_name VARCHAR,
    brand VARCHAR,
    product_description VARCHAR,
    product_attribute VARCHAR,
    product_details VARCHAR,
    product_category VARCHAR,
    price REAL,
    price_raw VARCHAR,
    overall_rating REAL,
    overall_num_rating INTEGER,
    last_updated TIMESTAMP(6)
)
WITH (
    format = 'PARQUET',
    partitioning = ARRAY['country', 'brand'],
    sorted_by = ARRAY['overall_num_rating', 'overall_rating']
)
""")

In [12]:
cur.close()
conn.close()